# Weather API 


In this NB we import the weather data, preprocess if is needed and save as a new Data Frame. 


### Historical Data 

For training and testing the model we use the historicla weather data related to our energy consumption data provided. 

For that, we'll use the API provided by [AT-Wetter](http://at-wetter.tk/index.php?men=api). We'll use this one, and not the OpenWeather API since if we want to get the historical data depth more than a week (we need a year) we'll have to use several queries. 


#### API 

This API don't need a Key or other autentication. It's located on the AT-Server in `http://at-wetter.tk/api/v1/.`
The instruction to use it are in their [website](http://at-wetter.tk/index.php?men=api) 

#### Example 

To query **all the available stations** we need a call following the format: 
`/api/v1/[field]/[YYYY-MM-DD]/[day count]` 

A query call to `http://at-wetter.tk/api/v1/t/2014-08-01/2`  returns Temperature Data for every available station for 2014-08-01, 2 days in the past. 



To query just **one specific stations** we use: 

`/api/v1/station/[station]/[field]|[all]/[YYYY-MM-DD]/[day count]`

This will return the `[field]` from the specified date `[YYYY-MM-DD]` and prints data `[day count]` days in the past for the specified stations `[station]`



### Calling the API


In [5]:
import requests as rq
import json
import datetime as dt


* See the available stations 

In [8]:
URL ="http://at-wetter.tk/api/v1/stations"
response = rq.request("GET", URL)
print(response.text)


'11157';'Aigen im Ennstal';'640';'m'
'11244';'Bad Gleichenberg';'280';'m'
'11101';'Bregenz';'424';'m'
'11190';'Eisenstadt';'184';'m'
'11';'Feuerkogel';'1618';'m'
'11155';'Feuerkogel';'1618';'m'
'11240';'Graz/Flughafen';'340';'m'
'11121';'Innsbruck';'579';'m'
'11331';'Klagenfurt/Flughafen';'447';'m'
'11012';'Kremsmünster';'383';'m'
'11012';'Kremsm�nster';'383';'m'
'11130';'Kufstein';'495';'m'
'11204';'Lienz';'659';'m'
'11010';'Linz/Hörsching';'298';'m'
'11010';'Linz/H�rsching';'298';'m'
'11171';'Mariazell';'866';'m'
'11126';'Patscherkofel';'2247';'m'
'11022';'Retz';'320';'m'
'11150';'Salzburg';'430';'m'
'11343';'Sonnblick';'3105';'m'
'11389';'St. Pölten';'270';'m'
'11389';'St. P�lten';'270';'m'
'11265';'Villacher Alpe';'2140';'m'
'11035';'Wien/Hohe Warte';'203';'m'
'11036';'Wien/Schwechat';'183';'m'



* Calculate de day count from Building Energy Data set 

We cold do it manually, but in order to automated all the steps we'll calculated with a python script 

In [72]:
import pandas as pd 
DataFrame = pd.read_excel("./DataSets/Building_energy.xlsx", header=1)
DataFrame.head()

#Calculate the Delta Time 

start_date = datetime.datetime.strptime(DataFrame['Time'][0], "%Y-%m-%d %H:%M:%S")
end_date = datetime.datetime.strptime(DataFrame['Time'][len(DataFrame)-1], "%Y-%m-%d %H:%M:%S")
day_count = end_date - start_date
print(day_count.days)


    


280


As the `[daycount]` field for API Query doesn't allow hours, if we have any hours of difference between the start day and the end day, we add an extra day. Then in the data frame matching we'll delethe the extra hours for the extra day. 

In [ ]:
if day_count.hours > 0: 
    day_count.days += 1 

* Download the temperature data

In [117]:
station = '11240' #GRAZ 
field = 't'  #Temperatuer
days_count = str(day_count.days)

URL = "http://at-wetter.tk/api/v1/station/"
URL += station 
URL += "/" + field
URL += "/" + str(end_date.year) + "-" + str(end_date.month) + "-" + str(end_date.day)
URL += "/" + days_count
response = rq.request("GET", URL)

print(URL)
#print(response.text)
type(response.text)

http://at-wetter.tk/api/v1/station/11240/t/2021-11-3/280


str

* Saving the data into a Data Frame and making its format equal to Energy Consumption data frame 

In [2]:
rows = response.text.splitlines()
df = pd.DataFrame(rows)
df.to_csv('./DataSets/tempDataSet.csv')

NameError: name 'response' is not defined

In [3]:
tempdf = pd.read_csv("./DataSets/tempDataSet.csv")
tempdf.head()

NameError: name 'pd' is not defined

### Forecast 
#### API 

We'll use python to request de dataset from the OpenWeather data base. The API  request call should be in this format: 

``` http://history.openweathermap.org/data/2.5/history/city?q={city ID},{country code}&type=hour&start={start}&end={end}&appid={API key}```

#### Usage 

Once we introduce the day that we want to predict we request for that day's forecast. If we have it, we arrange all the necessarry inputs for that day (temp, holiday, week day,etc) and we introduce them into the model. 

It's gonna be an input for the user to introduce the month, day, and hour. Then we check for the necessary data for that day (if it's a monday, a holiday, etc), we 